In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests


In [2]:
import requests
import re
from IPython.display import display

In [69]:
r = requests.get('https://raw.githubusercontent.com/jdh-observer/jdh001-L2gBr3BzwH8Z/main/DigitalHistoryRepresentations.ipynb')
notebook = r.json()
def flatten(t):
    return [item for sublist in t for item in sublist]

Get citations keys and listed authors from the notebook metadata:

In [85]:
citations = notebook.get('metadata').get('cite2c').get('citations')
citations_ids = [k for k in citations.keys()]

authors_citations_links = flatten([[(a,k) for a in x.get('author', [])] for (k,x) in citations.items()])
# print(citations_ids)
print(authors_citations_links[:1])

[({'family': 'Heiden', 'given': 'Serge'}, '6963300/3DVACENY')]


`authors_citations`is a list of tuples (author, reference) cited in the paper.
Let's see where the citations are looping the nootebook cells

In [108]:
# collect 
authors_citations_grouped_by_author = {}
for a,ref in authors_citations_links:
    author_name = f"{a.get('family')}-{a.get('given')}"
    if authors_citations_grouped_by_author.get(author_name, None) is None:
        authors_citations_grouped_by_author[author_name] = {
            'a': a,
            'refs':[], 
            'count_refs': 0,
            'count_citations': 0
        }
    authors_citations_grouped_by_author[author_name]['refs'].append(ref)
    authors_citations_grouped_by_author[author_name]['count_refs'] += 1 
# sort by number of publications
hall_of_fame = sorted(
    authors_citations_grouped_by_author.values(), 
    key=lambda x: x.get('count_refs'), reverse=True
)
print(hall_of_fame[:2])

[{'a': {'family': 'Zaagsma', 'given': 'Gerben'}, 'refs': ['6963300/7LBYNZC9', 'undefined'], 'count_refs': 2, 'count_citations': 0}, {'a': {'family': 'Beiner', 'given': 'Guy'}, 'refs': ['6963300/9A62ALCK', '6963300/TB6GZLEU'], 'count_refs': 2, 'count_citations': 0}]


In [109]:
cells = notebook.get('cells')


for cell in cells:
    tags = cell.get('metadata').get('tags', [])
    if 'hidden' in tags:
        continue
    source = ''.join(cell.get('source', []))
    
    if not source:
        continue
    # does it contains a cite2c marker?
    notes = re.findall(r'data-cite=\"([^\"]+)\"', source)
    if not notes:
        continue
    for note_key in notes:
        for author in hall_of_fame:
            if note_key in author['refs']:
                author['count_citations'] += 1

# sort
authors = sorted(hall_of_fame, key=lambda x: x.get('count_citations'), reverse=True)

for a in authors:
    name = f"{a.get('a').get('family')}, {a.get('a').get('given')}"
    print(f"{name} : {a.get('count_citations')} {a.get('refs')}")

Covington, Sarah : 10 ['6963300/TQU6YI5K']
Ó Siochrú, Micheál : 6 ['6963300/HYGCRXKH']
de Blacam, Aodh : 6 ['6963300/QTDMGGCU']
Perkins, Sonya : 5 ['6963300/GECT7PG7', '6963300/S8XC7HEE']
Elliott, Marianne : 3 ['6963300/RZUHQB3Z', '6963300/Z3UGXIQZ']
Ory, Pascal : 3 ['6963300/96SHBX6B']
J. S. Morrill,  : 3 ['6963300/FW8GA7FH']
Dillon, James : 3 ['6963300/WF75PUSN']
Beiner, Guy : 2 ['6963300/9A62ALCK', '6963300/TB6GZLEU']
MacManus, Francis : 2 ['6963300/A882G2RW', '6963300/WYAHHLCQ']
Revel, Jacques : 2 ['6963300/FDMFSTA2', '6963300/WC2BWFWY']
Delacroix, Christian : 2 ['6963300/PAHCHIIP', '6963300/SM32VD28']
Fairclough, Norman : 2 ['6963300/PQA2T3Z3', '6963300/RPJ4GP4E']
John Patrick Prendergast,  : 2 ['6963300/3E75G3KF']
'Ultach',  : 2 ['6963300/6W6VBX96']
Irish Propaganda Bureau,  : 2 ['6963300/9S7VRZAL']
Kalifa, Dominique : 2 ['6963300/B5F7ANP9']
Cunningham, John : 2 ['6963300/F7W7B4CY']
Barnard, Toby : 2 ['6963300/FSFUERFE']
Murray, Nora J. : 2 ['6963300/LS5CTR5E']
Venayre, Sylvain :